# GoogleLolaboratoryからboto3を使ってEC2にインスタンスを起動するサンプルnotebook

> Amazon Web Services (AWS)のPython向けSDKであるBotoをGoogleColaboratoryから使用し、EC2にネットワークを構築し、そこへインスタンスを起動するサンプルです。

- toc: true
- badges: ture
- comments: false
- categories: [jupyter,boto3]

# このnotebookは......

Amazon Web Services (AWS)のPython向けSDKであるBotoをGoogleColaboratoryから使用し、EC2にネットワークを構築し、そこへインスタンスを起動するサンプルです。

起動したインスタンスへはGoogleColaboratoryから **ではなく** PCからSSHでアクセスできるように設定します。

Botoの詳細は、 [公式ドキュメント](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html) を参照してください。特にEC2Clientについてはhttps://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/ec2.html を参照してください。


# 準備

## AWSアクセスキーを作成しておく

Botoを使用するには、事前にAWSのアクセスキーを用意する必要があります。

アクセスキー管理方法は様々ありますが、AWS IAMのユーザー https://console.aws.amazon.com/iam/home?region=us-west-2#/users からNotebook用のユーザーを作成する方法があります。万が一アクセスキーが漏れた場合に備えて、権限を最小限に、いつでも無効化できるように設定する必要があります。


## botoをインストールする

Boto公式ドキュメントの [Installation](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/quickstart.html#installation) を参考に、GoogleColaboratoryのラインタイム環境へ、boto3をインストールします。

In [ ]:
!pip install boto3

## botoのための認証資格情報を環境変数に設定する

Botoを使用するためには、認証資格情報をセットアップする必要があります。
このnotebookでは以下３つの環境変数を設定することにします。


1. AWS_ACCESS_KEY_ID
1. AWS_SECRET_ACCESS_KEY
1. AWS_DEFAULT_REGION

AWS_ACCESS_KEY_IDとAWS_SECRET_ACCESS_KEYには、AWSアクセスキーのアクセスキー IDとシークレットアクセスキーを設定します。

AWS_DEFAULT_REGIONには使用するリージョンを設定します。このnotebookでは東京リージョン( ap-northeast-1 )を使用します。


### アクセスキーIDを設定する

In [ ]:
import os
import getpass

os.environ['AWS_ACCESS_KEY_ID'] = getpass.getpass()

### シークレットアクセスキーを設定する

In [ ]:
os.environ['AWS_SECRET_ACCESS_KEY'] = getpass.getpass()

### デフォルトリージョンを設定する



In [ ]:
os.environ['AWS_DEFAULT_REGION'] = 'ap-northeast-1'

## pandasをインポートしておく

pandasを多用することになるので、インポートしておく

In [ ]:
import pandas as pd

## ec2のclientを作っておく

In [ ]:
import boto3
ec2_client = boto3.client('ec2')

# ネットワークを構築する

EC2インスタンスを起動するネットワークを構築します

## 新しいVPCを作る

### 現在のVPCのリストを確認する

In [ ]:
pd.DataFrame(ec2_client.describe_vpcs()['Vpcs'])

### 新しいVPCを作成する

今回は、`10.0.0.0/16`のCIDRブロックを持つVPCを作成する

In [ ]:
result = ec2_client.create_vpc(CidrBlock = '10.0.0.0/16',)

In [ ]:
result

作成したvpcの情報を`NEW_VPC`に入れておく

In [ ]:
NEW_VPC = result['Vpc']
NEW_VPC

### VPCのリストを確認する

新しく作成したVPCも見えるはず

In [ ]:
pd.DataFrame(ec2_client.describe_vpcs()['Vpcs'])

見えた

## サブネットを作成する

新しく作成したVPCにサブネットを作成する

### 現在のサブネットのリストを確認する

In [ ]:
subnets = ec2_client.describe_subnets()['Subnets']
pd.DataFrame(subnets)

新しく再生したVPCのサブネットは、

In [ ]:
list(filter(lambda x:x['VpcId'] == NEW_VPC['VpcId'], subnets))

当然、ない。

### 新しいVPCにサブネットを作成する

availability zoneは`ap=northeast-1a`, CIDRブロックは`10.0.0.0/24`にする


In [ ]:
result = ec2_client.create_subnet(
    AvailabilityZone = 'ap-northeast-1a',
    CidrBlock = '10.0.0.0/24',
    VpcId = NEW_VPC['VpcId']
)
result

In [ ]:
NEW_SUBNET = result['Subnet']
NEW_SUBNET

### サブネットのリストを確認する

新しく作成したサブネットも見えるはず

In [ ]:
subnets = ec2_client.describe_subnets()['Subnets']
pd.DataFrame(subnets)

In [ ]:
list(filter(lambda x:x['VpcId'] == NEW_VPC['VpcId'], subnets))

## サブネットにインスタンス起動時にパブリックIPを付与する設定を追加する

In [ ]:
response = ec2_client.modify_subnet_attribute(
    MapPublicIpOnLaunch={'Value': True},
    SubnetId=NEW_SUBNET['SubnetId']
)
response

In [ ]:
list(filter(lambda x:x['VpcId'] == NEW_VPC['VpcId'], ec2_client.describe_subnets()['Subnets']))

`MapPublicIpOnLaunch`の値がTrueになっている

## インターネットゲートウェイを作成する

### 現在のインターネットゲートウェイのリストを確認する

In [ ]:
internet_gateways = ec2_client.describe_internet_gateways()['InternetGateways']
pd.DataFrame(internet_gateways)

### 新しいインターネットゲートウェイを作成する

In [ ]:
result = ec2_client.create_internet_gateway()
result

In [ ]:
NEW_IG = result['InternetGateway']
NEW_IG

### インターネットゲートウェイのリストを確認する

In [ ]:
internet_gateways = ec2_client.describe_internet_gateways()['InternetGateways']
pd.DataFrame(internet_gateways)

## VPCにインターネットゲートウェイをアタッチする


### アタッチする

In [ ]:
result = ec2_client.attach_internet_gateway(
    InternetGatewayId = NEW_IG['InternetGatewayId'],
    VpcId = NEW_VPC['VpcId']
)

result

### インターネットゲートウェイの設定を確認する

In [ ]:
list(filter(lambda x:x['InternetGatewayId'] == NEW_IG['InternetGatewayId'], ec2_client.describe_internet_gateways()['InternetGateways']))

`Attachments`にアタッチしたVPCのIDが設定されている

## サブネットにインターネットゲートウェイをデフォルトゲートウェイにするルートを追加する

以下の順番で行います

1. 新しいルートテーブルを作成する
1. 作成したルートテーブルにインターネットゲートウェイをデフォルトゲートウェイにするルートを追加する
1. 作成したルートテーブルを作成したサブネットに関連づける

### 現在のルートテーブルのリストを確認する

In [ ]:
route_tables = ec2_client.describe_route_tables()['RouteTables']
pd.DataFrame(route_tables)

新しいVPCのルートテーブルを確認する

In [ ]:
route_table = list(filter(lambda x:x['VpcId'] == NEW_VPC['VpcId'], ec2_client.describe_route_tables()['RouteTables']))[0]
route_table

`Routes`に VPCのCidrブロック(10.0.0.0/16)宛のパケットを`local`、つまりVPC領域のルータへ転送する設定がされていることがわかる

### ルートテーブルを作成する

In [ ]:
response = ec2_client.create_route_table(
    VpcId = NEW_VPC['VpcId']
)
response

In [ ]:
NEW＿ROUTE_TABLE = response['RouteTable']
NEW＿ROUTE_TABLE

この時点で、`Routes`は既存のルートテーブルと同じになっている。また、`Associations`が空なので何にも関連づけられていない。

### 作成したルートテーブルに、全てのパケットをインターネットゲートウェイに転送するルートを作成する

In [ ]:
response = ec2_client.create_route(
    DestinationCidrBlock = '0.0.0.0/0',
    GatewayId = NEW_IG['InternetGatewayId'],
    RouteTableId = NEW＿ROUTE_TABLE['RouteTableId']
)
response

新しいルートテーブルの設定を確認する

In [ ]:
route_table = list(filter(lambda x:x['RouteTableId'] == NEW＿ROUTE_TABLE['RouteTableId'], ec2_client.describe_route_tables()['RouteTables']))[0]
route_table

`Routes`にDestinationCidrBlockが0.0.0.0/0の設定が追加されている

### 新しいルートテーブルをサブネットに関連づける

In [ ]:
response = ec2_client.associate_route_table(
    RouteTableId = NEW＿ROUTE_TABLE['RouteTableId'],
    SubnetId = NEW_SUBNET['SubnetId']
)
response

新しいルートテーブルの設定を確認する

In [ ]:
route_table = list(filter(lambda x:x['RouteTableId'] == NEW＿ROUTE_TABLE['RouteTableId'], ec2_client.describe_route_tables()['RouteTables']))[0]
route_table

`Associations`が設定されていることがわかる

# サーバを構築する

## セキュリティグループを作成する

### 現在のセキュリティグループのリストを確認する

新しくVPCを作成するとデフォルトのセキュリティグループが作成されるので確認する

In [ ]:
list(filter(lambda x:x['VpcId'] == NEW_VPC['VpcId'], ec2_client.describe_security_groups()['SecurityGroups']))

### 新しいセキュリティグループを作成する

In [ ]:
response = ec2_client.create_security_group(
    Description = 'from-colab',
    GroupName = 'from-colab',
    VpcId = NEW_VPC['VpcId']
)
response

In [ ]:
NEW_SECURITY_GROUP_ID = response['GroupId']
NEW_SECURITY_GROUP_ID

### 作成したセキュリティグループの設定を確認しておく

In [ ]:
list(filter(lambda x:x['GroupId'] == NEW_SECURITY_GROUP_ID, ec2_client.describe_security_groups()['SecurityGroups']))

## セキュリティグループにsshアクセス用の設定を追加する

### インスタンスにSSHでアクセスするPCのIPアドレスを設定する

AWSコンソールから設定する場合は 'マイ IP' を選択すれば自分のPCのIPアドレスを設定できましたが、boto3から設定する場合は自分のIPアドレスを値として設定する必要があります

In [ ]:
myip = 'xxx.xxx.xxx.xxx/xx'

### セキュリティグループに設定を追加する

In [ ]:
response = ec2_client.authorize_security_group_ingress(
    GroupId = NEW_SECURITY_GROUP_ID,
    IpPermissions = [{
        'FromPort': 22,
        'IpProtocol': 'tcp',
        'IpRanges': [{
            'CidrIp': myip,
            'Description': 'SSH access from my ip'
        }],
        'ToPort': 22
    }]
)
response

### セキュリティグループの設定を確認しておく

In [ ]:
list(filter(lambda x:x['GroupId'] == NEW_SECURITY_GROUP_ID, ec2_client.describe_security_groups()['SecurityGroups']))

`IpPermissions`に設定が追加されている

## キーペアを作成する

### 秘密鍵の保存先としてGoogleDriveをマウントする

Google ドライブをマウントするには、次のセルを実行してください。  
実行すると、ランタイム環境がGoogleDriveへアクセスするための権限を要求されます。

1. Go to this URL in a browser: https://accounts.google.com/o/oauth2/<以下略> という形式のリンクが表示されるので、マウスでクリックして開いてください
2. (必要に応じてGoogleアカウントへのログインと)開いた先でアクセス権限を要求されるので、承認してください  承認すると認証コードが表示されるのでコピーします。
3. Enter your authorization code: と表示されているテキストボックスへ認証コードを入力しEnterを押下してください.  Mounted at /content/drive と表示されれば成功です


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### キーペアを作成する

In [ ]:
response = ec2_client.create_key_pair(
   KeyName = 'from-colab'
)
response

In [ ]:
NEW_KEY_NAME = response['KeyName']
NEW_KEY_NAME

### 秘密鍵を保存する

GoogleDriveの"マイドライブ"保存します。
保存後、SSHアクセスするPCへ移動させてください。

In [ ]:
file_path = '/content/drive/MyDrive/{}.pem'.format(NEW_KEY_NAME)
with open(file_path, mode='w') as f:
  f.write(response['KeyMaterial'])

## インスタンスを起動する

### インスタンス名を設定する

In [ ]:
instance_name = 'from-colab'

### イメージidを設定する

インスタンスにインストールするイメージのidを設定します。  
今回は`amzn2-ami-hvm-2.0.20210427.0-x86_64-gp2`を使います。

// 本当は`describe_images`メソッドを使って探したいところですが、  
// イメージの数が多いので単純に探すと時間がかかり、絞り込みも大変なので  
// awsコンソールのec2からインスタンスの起動を選び、ステップ1: Amazon マシンイメージ(AMI)で確認するのが簡単です。

In [ ]:
image_id = 'ami-0ca38c7440de1749a'

image_idでimageが見つかることを確認しておく

In [ ]:
ec2_client.describe_images(
    ImageIds = [image_id]
)

### インスタンスを起動する

In [ ]:
responce = ec2_client.run_instances(
    ImageId = image_id,
    MinCount = 1,
    MaxCount=1,
    InstanceType='t2.micro',
    KeyName = NEW_KEY_NAME,
    SecurityGroupIds = [NEW_SECURITY_GROUP_ID],
    SubnetId = NEW_SUBNET['SubnetId'],
    TagSpecifications = [{
        'ResourceType': 'instance',
        'Tags': [
            {'Key': 'Name', 'Value': instance_name}
        ]
    }]
)

In [ ]:
NEW_INSTANCES = responce['Instances']
NEW_INSTANCES

### 起動したインスタンスのパブリックIPを確認する

`run_instances`のタイミングではパブリックIPは設定されていないので、起動が終わるのを待ってから詳細情報を取得し、そこからパブリックIPを得る

In [ ]:
response = ec2_client.describe_instances(
    InstanceIds = list(map(lambda x:x['InstanceId'], NEW_INSTANCES))
)
response

In [ ]:
pd.DataFrame([
  {
        'InstanceId': insntance['InstanceId'],
        'InsntanceNmae': list(map(lambda x:x['Value'], filter(lambda x:x['Key'] == 'Name', insntance['Tags'])))[0],
        'KeyName': insntance['KeyName'],
        'PublicIpAddress': insntance['PublicIpAddress'],
  } for reservation in response['Reservations'] for insntance in reservation['Instances'] 
])

### ssh接続確認

上で出力した情報を参考PCから起動したインスタンスへsshで接続できることを確認します。

確認できたら、boto3を使ったEC2インスタンスの起動は完了です。


# 後片付け

このnotebookはサンプルなので、作成したリソースを削除します。  
もし、作成したリソースを使い続けたい場合はこのセクションはスキップしてください

## インスタンスを削除する

In [ ]:
response = ec2_client.terminate_instances(
    InstanceIds = list(map(lambda x:x['InstanceId'], NEW_INSTANCES))
)
response

## キーペアを削除する

In [ ]:
response = ec2_client.delete_key_pair(KeyName=NEW_KEY_NAME)
response

## セキュリティグループを削除する

In [ ]:
response = ec2_client.delete_security_group(GroupId=NEW_SECURITY_GROUP_ID)
response

## ルートテーブルを削除する

In [ ]:
route_table = list(filter(lambda x:x['RouteTableId'] == NEW＿ROUTE_TABLE['RouteTableId'], ec2_client.describe_route_tables()['RouteTables']))[0]
route_table

### ルートテーブルの関連付けを解除する

In [ ]:
for association in route_table['Associations']:
  response = ec2_client.disassociate_route_table(
      AssociationId = association['RouteTableAssociationId']
  )
  print(response)

### ルートテーブルを削除する

In [ ]:
responce = ec2_client.delete_route_table(
    RouteTableId = NEW＿ROUTE_TABLE['RouteTableId']
)
responce

## インターネットゲートウェイを削除する

## インターネットゲートウェイをVPCからデタッチする

In [ ]:
responce = ec2_client.detach_internet_gateway(
    InternetGatewayId = NEW_IG['InternetGatewayId'],
    VpcId = NEW_VPC['VpcId']
)
responce

### インターネットゲートウェイを削除する

In [ ]:
responce = ec2_client.delete_internet_gateway(
    InternetGatewayId = NEW_IG['InternetGatewayId']
)
responce

## サブネットを削除する

In [ ]:
response = ec2_client.delete_subnet(
    SubnetId = NEW_SUBNET['SubnetId']
)
response

## VPCを削除する

In [ ]:
response = ec2_client.delete_vpc(
    VpcId = NEW_VPC['VpcId']
)
response

以上